# How Stochastic Gradient Descent code works

## From Rashka's ML book, 2017

### Initially, how to set up classes

Rashka uses:

In [1]:
class MyClass(object):
    pass

This makes the code Python agnostic between Python 2 and Python 3. For my purposes just use:

In [2]:
class MyClass:
    pass

**Rule of thumb:** don't introduce new attribute outside \__init__

**self** refers to the instances (objects) of the class

| Non-OOL | OOL |
| :---: | :---: |
| variable | attribute |
| function | method |


**Different types of attributes/methods:**<br>Static attibutes/methods, are the same in all instances of the class:

In [10]:
class Car:
    wheels = 4
    
nissan = Car()
VW = Car()
print((nissan.wheels, VW.wheels))

(4, 4)


The 'normal' everyday methods are Instance methods, and call (self):

In [13]:
class Car:
    wheels = 4
    
    def __init__(self, price_range):
        self.price_range = (price_range)
        
nissan = Car('Economy')
VW = Car('Premium Economy')

print(nissan.price_range)
print(VW.price_range)

Economy
Premium Economy


Different methods are shown below. Class method and static method have decorator labels. These are nicely explained at:<br>
https://realpython.com/primer-on-python-decorators/

In [24]:
class Car:
    wheels = 4
    
    def __init__(self, price_range):
        self.price_range = (price_range)
    
    def method(self):
        return 'instance method called', self

    @classmethod
    def classmethod(cls):
        return 'class method called', cls

    @staticmethod
    def staticmethod():
        return 'static method called'
    


In [26]:
mercedes = Car('Premium')
mercedes.method()

('instance method called', <__main__.Car at 0x10ffcd3c8>)

In [27]:
mercedes.classmethod()

('class method called', __main__.Car)

## Playing with methods in a class

### See Rashka's own explanation at:

https://github.com/rasbt/python-machine-learning-book/blob/master/faq/underscore-convention.md

Basically, a trailing underscore_: sklearn convention for something which is estimated; leading \_underscore is a 'private' attribute inside the class (are helpers).


This works inside the \__init__:

In [78]:
# create a matrix inside the class
class UnderstandCode:
    def __init__(self, m, n, seed):
        self.m = m
        self.n = n
        self.randomseed = seed
             
        def creatematrix(self):
            matrix = []
            for _ in range(self.n):
                row = []
                for _ in range(self.m):
                    row.append(self.randomseed)
                matrix.append(row)
            return matrix
                
        self.matrix = creatematrix(self)
    
    def helper(self):
        print(self.matrix)
        
z = UnderstandCode(3,4,0)
z.helper()

[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]]


I had a **HUGE MISUNDERSTANDING** reading Rashka's code!

For instance, in his SGD Adaline, in *fit method* it envokes self.\_initialize_weights(X.shape[1]). This isn't an attribute, it tells the \_initialize_weights function to happen, causing the self.w_ attribute to be created. self.w_ is not in \__init__, as it isn't needed until the fit is called.


In [125]:
# introduce random numbers
import numpy as np
rgen = np.random.RandomState(4)

# what does the self.w_ look like?
w_ = rgen.normal(loc=0.0, scale=0.01, size=4)
print("w_:", w_)

# take the matrix creator outide the class
def creatematrix(m, n, seed):
    matrix = []
    rgen = np.random.RandomState(seed)
    for _ in range(n):
        row = []
        for _ in range(m):
            row.append(rgen.randint(0,9))
        matrix.append(row)
    return matrix

class UnderstandCode:
    def __init__(self, seed):
        self.rgen = np.random.RandomState(seed)
        
    def fit(self, matrix):
        self._initialize_weights(matrix)
        
    def _initialize_weights(self, m):
        """Initialize weights to small random numbers"""
        self.w_ = self.rgen.normal(loc=0.0, scale=0.01, size=m)
    
    def something(self, n):
        self._anything(n)
        # ^ I am asking _anytthing function to happen. 
        # If invoked I can print the .a_ attribute (but not earlier).
    
    def _anything(self, n):
        self.a_ = [0] * n

    def helper(self):
        print("self.w_", self.w_)
        print("self.a_", self.a_)
        
data = creatematrix(3, 4, 0)
# print(data)

uc01 = UnderstandCode(0)
uc01.fit(len(data[0]))
print("w_", uc01.w_)
print()

# Created another 'pair' (something and _anything) and do the same thing
uc02 = UnderstandCode(1)
uc02.fit(len(data[0]))
uc02.something(11)

# see what happened with helper function in class
uc02.helper()

w_: [ 0.00050562  0.00499951 -0.00995909  0.00693599]
w_ [0.01764052 0.00400157 0.00978738]

self.w_ [ 0.01624345 -0.00611756 -0.00528172]
self.a_ [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


Rashka's Github simpler examples:

In [126]:
class MyEstimator():
    def __init__(self):
        self.param = 1.0

    def fit(self):
        self.fit_param_ = [0.1] * 5
    
z = MyEstimator()
print(z.param)
z.fit()
print(z.fit_param_)

1.0
[0.1, 0.1, 0.1, 0.1, 0.1]


In [127]:
class MyClass(object):
    def __init__(self, param='some_value'):
        pass

    def public(self):
        'User, this public method is for you!'
        return 'public method'

    def _indicate_private(self):
        return 'private method'

    def __pseudo_private(self):
        return 'really private method'

In [128]:
MyObj = MyClass()
MyObj.public()

'public method'

In [129]:
MyObj._indicate_private()


'private method'